In [2]:
!conda install pytorch=1.10.0 torchvision torchaudio=0.10.0 cudatoolkit=11.1 -c pytorch -c conda-forge -y


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cudatoolkit=11.1
    - pytorch=1.10.0
    - torchaudio=0.10.0
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.1.1         |      ha002fc5_10        1.20 GB  conda-forge
    ffmpeg-4.2                 |       h167e202_0        80.2 MB  conda-forge
    gnutls-3.6.13              |       h85f3911_1         2.0 MB  conda-forge
    lame-3.100                 |    h7f98852_1001         496 KB  conda-forge
    libiconv-1.17              |       h166bdaf_0         1.4 MB  conda-forge

In [3]:
%%bash
#use the official code. Note: run-librespeech.sh, requirements.txt, train.py and run.sh need minor modification before the training can run on SageMaker
mkdir /root/wenet-source
cd /root/wenet-source
git clone https://github.com/wenet-e2e/wenet.git
cd wenet
git stash
git checkout v2.2.0
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 25.1 MB/s eta 0:00:00
  Using cached tensorboardX-2.6-py2.py3-none-any.whl (114 kB)
  Using cached typeguard-3.0.2-py3-none-any.whl (30 kB)
  Using cached TextGrid-1.5-py3-none-any.whl (10.0 kB)
  Using cached flake8-3.8.2-py2.py3-none-any.whl (72 kB)
  Using cached flake8_bugbear-23.3.12-py3-none-any.whl (28 kB)
  Using cached flake8_comprehensions-3.11.1-py3-none-any.whl (7.7 kB)
  Using cached flake8_executable-2.1.3-py3-none-any.whl (35 kB)
  Using cached flake8_pyi-20.5.0-py36-none-any.whl (9.6 kB)
  Using cached pycodestyle-2.6.0-py2.py3-none-any.whl (41 kB)
  Using cached pyflakes-2.2.0-py2.py3-none-any.whl (66 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 45.4 MB/s eta 0:00:00
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.wh

In [ ]:
%%bash
#or use the folked one. All scripts prepared
mkdir /root/wenet-source
cd /root/wenet-source
git clone https://github.com/goriri/wenet220.git
cd wenet
pip install -r requirements.txt

In [11]:
# install gcc and other stuff if you are using the Data Science Kernel
!apt-get -y update
!apt-get -y install build-essential


Hit:1 http://security.debian.org/debian-security buster/updates InRelease
Hit:2 http://deb.debian.org/debian buster InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu cpp cpp-8 dirmngr
  dpkg-dev fakeroot g++ g++-8 gcc gcc-8 gnupg gnupg-l10n gnupg-utils gpg
  gpg-agent gpg-wks-client gpg-wks-server gpgconf gpgsm gpgv
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libalgorithm-merge-perl
  libasan5 libassuan0 libatomic1 libbinutils libcc1-0 libdpkg-perl libfakeroot
  libfile-fcntllock-perl libgcc-8-dev libisl19 libitm1 libksba8
  liblocale-gettext-perl liblsan0 libmpc3 libmpfr6 libmpx2 libnpth0
  libquadmath0 libstdc++-8-dev libtsan0 libubsan1 make pinentry-curses
Suggested packages:
  binutils-doc cpp-doc gcc-8-locales dbus-user-sessio

In [12]:
# install flac 
!wget https://downloads.xiph.org/releases/flac/flac-1.3.2.tar.xz
!xz -d flac-1.3.2.tar.xz && tar -xvf flac-1.3.2.tar --no-same-owner && cd flac-1.3.2 && ./configure --prefix=/usr --disable-thorough-tests && make && make install

flac-1.3.2/
flac-1.3.2/compile
flac-1.3.2/aclocal.m4
flac-1.3.2/missing
flac-1.3.2/examples/
flac-1.3.2/examples/Makefile.lite
flac-1.3.2/examples/Makefile.am
flac-1.3.2/examples/README
flac-1.3.2/examples/cpp/
flac-1.3.2/examples/cpp/Makefile.am
flac-1.3.2/examples/cpp/encode/
flac-1.3.2/examples/cpp/encode/file/
flac-1.3.2/examples/cpp/encode/file/example_cpp_encode_file.vcxproj.filters
flac-1.3.2/examples/cpp/encode/file/Makefile.lite
flac-1.3.2/examples/cpp/encode/file/main.cpp
flac-1.3.2/examples/cpp/encode/file/example_cpp_encode_file.vcxproj
flac-1.3.2/examples/cpp/encode/file/Makefile.am
flac-1.3.2/examples/cpp/encode/file/Makefile.in
flac-1.3.2/examples/cpp/encode/file/example_cpp_encode_file.vcproj
flac-1.3.2/examples/cpp/encode/Makefile.am
flac-1.3.2/examples/cpp/encode/Makefile.in
flac-1.3.2/examples/cpp/Makefile.in
flac-1.3.2/examples/cpp/decode/
flac-1.3.2/examples/cpp/decode/file/
flac-1.3.2/examples/cpp/decode/file/example_cpp_decode_file.vcxproj.filters
flac-1.3.2/exam

configure: WARNING: *** XMMS >= 0.9.5.1 not installed - XMMS support will not be built
configure: WARNING: *** Ogg development enviroment not installed - Ogg support will not be built
bitreader.c: In function ‘crc16_update_word_’:
bitreader.c:122:16: warning: this statement may fall through [-Wimplicit-fallthrough=]
   case  0: crc = FLAC__CRC16_UPDATE((unsigned)(word >> 24), crc);
bitreader.c:123:3: note: here
   case  8: crc = FLAC__CRC16_UPDATE((unsigned)((word >> 16) & 0xff), crc);
   ^~~~
bitreader.c:123:16: warning: this statement may fall through [-Wimplicit-fallthrough=]
   case  8: crc = FLAC__CRC16_UPDATE((unsigned)((word >> 16) & 0xff), crc);
bitreader.c:124:3: note: here
   case 16: crc = FLAC__CRC16_UPDATE((unsigned)((word >> 8) & 0xff), crc);
   ^~~~
bitreader.c:124:16: warning: this statement may fall through [-Wimplicit-fallthrough=]
   case 16: crc = FLAC__CRC16_UPDATE((unsigned)((word >> 8) & 0xff), crc);
bitreader.c:125:3: note: here
   case 24: br->read_crc16 = FLAC

In [2]:
%cd "/root/wenet-source/wenet/examples/librispeech/s0/"
%env DATA_DIR=/root/librispeech/data/training


/root/wenet-source/wenet/examples/librispeech/s0
env: DATA_DIR=/root/wenet/data/training


In [6]:
# prepare dataset
!chmod 777 *.sh
!cp run.sh run-bak.sh
#mkdir export
!mkdir -p $DATA_DIR

In [ ]:
%%bash
#Data download
./run.sh --stage -1 --stop_stage -1 --datadir $DATA_DIR

In [8]:
%%bash
# Data preparep
./run.sh --stage 0 --stop_stage 0 --datadir $DATA_DIR

Process is interrupted.


In [4]:
#如果import失败，Kernel->Restart Kernel
import torch
print(torch.__version__)

1.10.0


In [7]:
%%bash
# Feature Generation. Using the SageMaker Pytorch kernel will fail at this step for some unknown reason
./run.sh --stage 1 --stop_stage 1 --datadir $DATA_DIR

#chmod -R 777 data

stage 1: Feature Generation
using resample and new sample rate is 16000
dictionary: data/lang_char/train_960_unigram5000_units.txt


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
processed 1000 wavs, 1227475 frames
processed 2000 wavs, 2454593 frames
processed 3000 wavs, 3677168 frames
processed 4000 wavs, 4914155 frames
processed 5000 wavs, 6160107 frames
processed 6000 wavs, 7433925 frames
processed 7000 wavs, 8661841 frames
processed 8000 wavs, 9885348 frames
processed 9000 wavs, 11141552 frames
processed 10000 wavs, 12380723 frames
processed 11000 wavs, 13609148 frames
processed 12000 wavs, 14857279 frames
processed 13000 wavs, 16092972 frames
processed 14000 wavs, 17332808 frames
processed 15000 wavs, 185690

In [8]:
%%bash
./run.sh --stage 2 --stop_stage 2 --datadir $DATA_DIR
    
#chmod -R 777 data

dictionary: data/lang_char/train_960_unigram5000_units.txt
stage 2: Dictionary and Json Data Preparation
5001 data/lang_char/train_960_unigram5000_units.txt


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=data/lang_char/input.txt --vocab_size=5000 --model_type=unigram --model_prefix=data/lang_char/train_960_unigram5000 --input_sentence_size=100000000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/lang_char/input.txt
  input_format: 
  model_prefix: data/lang_char/train_960_unigram5000
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 100000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 

In [9]:
%%bash
./run.sh --stage 3 --stop_stage 3 --datadir $DATA_DIR
    
#chmod -R 777 data

dictionary: data/lang_char/train_960_unigram5000_units.txt
Prepare data, prepare required format


In [10]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()

bucket = sess.default_bucket() 
bucket

'sagemaker-us-east-1-348052051973'

In [20]:
#use the default bucket or another bucket of your choice to replace the BUCKET here
%env BUCKET=sagemaker-us-east-1-348052051973
%env PREFIX=wenet220

env: BUCKET=sagemaker-us-east-1-348052051973
env: PREFIX=wenet220


In [18]:
%%bash
#Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)
for filename in data/*/data.list; do
    echo $filename
    sed -i 's#/root/librispeech#/opt/ml/input#g' $filename
done

data/dev/data.list
data/dev_clean/data.list
data/dev_other/data.list
data/test_clean/data.list
data/test_other/data.list
data/train_960/data.list


In [ ]:
%%bash
#upload dataset+etc to s3 bucket. Librispeech doesn't have the exp folder.
#aws s3 cp --recursive  exp s3://$BUCKET/$PREFIX/exp
aws s3 cp --recursive  data s3://$BUCKET/$PREFIX/data
#make sure directory data/train_960 exist


In [2]:
%%markdown
We could use terminal with following commands to speed up copying to s3. Replace sagemaker-us-east-1-348052051973 with your owne s3 bucket
nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/dev-clean s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/dev-clean --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/dev-other s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/dev-other --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/test-clean s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/test-clean --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/test-other s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/test-other --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/train-clean-100 s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/train-clean-100 --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/train-clean-360 s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/train-clean-360 --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/train-other-500 s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/train-other-500 --only-show-errors &

We could use terminal with following commands to speed up copying to s3. Replace sagemaker-us-east-1-348052051973 with your owne s3 bucket
nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/dev-clean s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/dev-clean --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/dev-other s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/dev-other --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/test-clean s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/test-clean --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/test-other s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/test-other --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/train-clean-100 s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/train-clean-100 --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/train-clean-360 s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/train-clean-360 --only-show-errors &

nohup aws s3 sync ~/librispeech/data/training/LibriSpeech/train-other-500 s3://sagemaker-us-east-1-348052051973/wenet220/export/LibriSpeech/train-other-500 --only-show-errors &
